In [6]:
import numpy as np
import cv2
import glob
import os

# 1. Camera Calibration
checkerboardsize = (9, 7)
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)

objp = np.zeros((checkerboardsize[0] * checkerboardsize[1], 3), np.float32)
objp[:, :2] = np.mgrid[0 : checkerboardsize[0], 0 : checkerboardsize[1]].T.reshape(
    -1, 2
)

objpoints = []
imgpoints = []

base_path = "/home/rafay/Documents/rafay/MyFolder_other_stuff/Educational/Vibot/M1/CV_fofi/cv_project_yohan/stereo_reconstruction_cv"
images = glob.glob("calibration_data_logitech_3840x2160/*.jpg")
images = sorted([os.path.join(base_path, img_path) for img_path in images])
annotation_dir = "chessboard_corners"
save_chessboard_corner_ann = True
if not os.path.exists(annotation_dir) and save_chessboard_corner_ann:
    os.mkdir(annotation_dir)


for fname in images:
    img = cv2.imread(fname)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    ret, corners = cv2.findChessboardCorners(
        gray,
        checkerboardsize,
        cv2.CALIB_CB_ADAPTIVE_THRESH
        + cv2.CALIB_CB_FAST_CHECK
        + cv2.CALIB_CB_NORMALIZE_IMAGE,
    )

    if ret == True:
        objpoints.append(objp)
        corners2 = cv2.cornerSubPix(gray, corners, (11, 11), (-1, -1), criteria)
        imgpoints.append(corners2)
        if save_chessboard_corner_ann:
            fname_ = (
                os.path.join(base_path, annotation_dir)
                + "/"
                + fname.split("/")[-1].split(".")[0]
                + "corner_plot.jpg"
            )
            cv2.drawChessboardCorners(img, checkerboardsize, corners2, ret)
            cv2.imwrite(fname_, img)

ret, cameraMatrix, dist, rvec, tvec = cv2.calibrateCamera(
    objpoints, imgpoints, gray.shape[::-1], None, None
)

# Compute reprojection error
reprojection_error = 0
for i in range(len(objpoints)):
    imgpoints2, _ = cv2.projectPoints(
        objpoints[i], rvec[i], tvec[i], cameraMatrix, dist
    )
    error = cv2.norm(imgpoints[i], imgpoints2, cv2.NORM_L2) / len(imgpoints2)
    reprojection_error += error
reprojection_error /= len(objpoints)
print("Reprojection error:", reprojection_error)
print("Camera Calibrated:   ", ret)
print("Camera Matrix:   ", cameraMatrix)
print("Distortion Parameters:   ", dist)
print("Rotation Vector: ", rvec)
print("Translation Vector: ", tvec)

Reprojection error: 0.14875925765044806
Camera Calibrated:    1.3396757464751792
Camera Matrix:    [[2.25370759e+03 0.00000000e+00 1.92969309e+03]
 [0.00000000e+00 2.24471892e+03 1.05763445e+03]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00]]
Distortion Parameters:    [[ 2.08993190e-01 -5.57647266e-01 -7.19877887e-06  5.16897145e-04
   3.81154053e-01]]
Rotation Vector:  (array([[ 0.0423316 ],
       [-0.01053158],
       [ 0.0203875 ]]), array([[-0.0618124 ],
       [-0.07042272],
       [ 1.55299835]]), array([[ 0.0633967 ],
       [-0.09186907],
       [ 0.79679293]]), array([[ 0.04796078],
       [-0.21691397],
       [-0.02970658]]), array([[ 0.23029487],
       [-0.21473849],
       [-0.28463919]]), array([[-0.08643709],
       [ 0.2049388 ],
       [ 0.19885457]]), array([[-0.28777851],
       [-0.05864591],
       [-0.00549516]]), array([[ 0.18931259],
       [-0.05374829],
       [ 0.11010374]]), array([[ 0.36772746],
       [-0.09665969],
       [ 0.1739259 ]]), array([[-0.50